# 개요
의사가 되기 위해서는 환자 노트를 작성하는 시험을 본다.
롤 플레이처럼 환자역할을 하는 사람(특정 질병에 대해서 묘사하도록 훈련된사람)에게 증상을 듣고 환자 노트를 작성하는 방식으로 시험을 본다.
환자 노트를 작성하는 **시험** 을 본다는 의미는 -> 누군가 채점을 해야한다는 것..

이런 시험은 채점이 정말 오래걸리고 힘들다. 돈도 많이든다. 따라서 자연어처리를 통해서 해결해보려고 여러가지 노력이 있었지.
하지만 이 마저도 여의치않아. 표현이 너무 다양하기 떄문에
예를들어 "활동을 안하고싶어한다"와 "이제 테니스를 안칠꺼다" 는 같은 뜻이 될 수 있다
또, "손이차고, 머리가 빠지고, 심장이 뛰고 등등등"라는 문장은 "갑상선에 trouble이 있음" 으로 표현되기도 한다.

우리는 특정 증상과 그를 표현하는 다양한 문장들을 매핑하는 솔루션을 만들어야함
ex) "식욕 떨어짐" == "식욕 없다", "옷이 점점 루즈핏이 된다" and etc

# 데이터 description

## 중요한 용어
- Clinical Case: 표준화된 환자가 테스트보는사람에게 제공되는 시나리오. 10개의 사례가 있음
- Patient Note: 환자와 관련된 중요한 정보 디테일하게 작성한거. (몸 검사 및 인터뷰)
- Feature: clinically 한 개념. rublic은 각 케이스와 관련이 있는 주요 개념을 설명함.

## training Data
- patient_notes.csv - 40000개의 환자노트 기록 모음. 이중에 일부만 annotated features를 갖고있음. 아 어노테이션 빼고 unsupervised learning 하고싶어할수도있음. 테스트셋에있는 환자노트는 public version에 없습니다.(당연)
  - pn_num - 환자노트 id
  - case_num - 환자노트의 clinical case의 id
  - pn_history - 시험본사람이 기록한 text

- features.csv - 각 clinical case의 채점기준표.
  - feature_num - feature의 id
  - case_num - clinical case의 id.
  - feature_text - 특징들의 description

- train.csv - 환자노트 1000개의 특징 annotation. 각 10개에 100개씩 있음.
  - id - id. pn_num + feature_num 을 pair로 해서 만들어짐
  - pn_num - 환자노트id.
  - feature_num - feature id
  - case_num - case num
  - annotation - feature를 가리키는 환자노트의 text. 하나의 노트에서 feature 하나가 여러번 나올 수 있음.
  - location - annotation의 idx 범위. 세미콜론(;)으로 구문되며 from,to 로 되어있는 듯?
  
## Example Test Data
테스트데이터 넣어줌
- test.csv - 
- sample_submission.csv - 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# import

In [ ]:
# import
import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import gc
from tqdm.notebook import tqdm
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from itertools import chain
import warnings
import json
from ast import literal_eval
import spacy

import scipy as sp

from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.metrics import precision_recall_fscore_support

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

import re

warnings.filterwarnings('ignore')

# Library

In [ ]:
# Library
# Transformer
os.system('pip uninstall -y transformers')
os.system('python -m pip install --no-index --find-links=../input/my-whl transformers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device :", device)

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='../input/huggingface-roberta-variants/roberta-large/roberta-large')
unmasker("Paris is the <mask> of France.")

# EDA
데이터를 한번씩 까봐야겠음

In [ ]:
patient_notes = pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')
patient_notes

In [ ]:
features = pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv')
features

In [ ]:
train = pd.read_csv('../input/nbme-score-clinical-patient-notes/train.csv')
train

In [ ]:
print(train.loc[0])
print(train.loc[338])
print(train.loc[338, "annotation"])
print(train.loc[338, "location"])

print(train.loc[655])
print(train.loc[655, "annotation"])
print(train.loc[655, "location"])

In [ ]:
# incorrect annotation
train.loc[338, 'annotation'] = "['father heart attack']"
train.loc[338, 'location'] = "['764 783']"

train.loc[621, 'annotation'] = "['for the last 2-3 months']"
train.loc[621, 'location'] = "['77 100']"

train.loc[655, 'annotation'] = "['no heat intolerance', 'no cold intolerance']"
train.loc[655, 'location'] = "['285 292;301 312', '285 287;296 312']"

train.loc[1262, 'annotation'] = "['mother thyroid problem']"
train.loc[1262, 'location'] = "['551 557;565 580']"

train.loc[1265, 'annotation'] = "['felt like he was going to \"pass out\"']"
train.loc[1265, 'location'] = "['131 135;181 212']"

train.loc[1396, 'annotation'] = "['stool , with no blood']"
train.loc[1396, 'location'] = "['259 280']"

train.loc[1591, 'annotation'] = "['diarrhoe non blooody']"
train.loc[1591, 'location'] = "['176 184;201 212']"

train.loc[1615, 'annotation'] = "['diarrhea for last 2-3 days']"
train.loc[1615, 'location'] = "['249 257;271 288']"

train.loc[1664, 'annotation'] = "['no vaginal discharge']"
train.loc[1664, 'location'] = "['822 824;907 924']"

train.loc[1714, 'annotation'] = "['started about 8-10 hours ago']"
train.loc[1714, 'location'] = "['101 129']"

train.loc[1929, 'annotation'] = "['no blood in the stool']"
train.loc[1929, 'location'] = "['531 539;549 561']"

train.loc[2134, 'annotation'] = "['last sexually active 9 months ago']"
train.loc[2134, 'location'] = "['540 560;581 593']"

train.loc[2191, 'annotation'] = "['right lower quadrant pain']"
train.loc[2191, 'location'] = "['32 57']"

train.loc[2553, 'annotation'] = "['diarrhoea no blood']"
train.loc[2553, 'location'] = "['308 317;376 384']"

train.loc[3124, 'annotation'] = "['sweating']"
train.loc[3124, 'location'] = "['549 557']"

train.loc[3858, 'annotation'] = "['previously as regular', 'previously eveyr 28-29 days', 'previously lasting 5 days', 'previously regular flow']"
train.loc[3858, 'location'] = "['102 123', '102 112;125 141', '102 112;143 157', '102 112;159 171']"

train.loc[4373, 'annotation'] = "['for 2 months']"
train.loc[4373, 'location'] = "['33 45']"

train.loc[4763, 'annotation'] = "['35 year old']"
train.loc[4763, 'location'] = "['5 16']"

train.loc[4782, 'annotation'] = "['darker brown stools']"
train.loc[4782, 'location'] = "['175 194']"

train.loc[4908, 'annotation'] = "['uncle with peptic ulcer']"
train.loc[4908, 'location'] = "['700 723']"

train.loc[6016, 'annotation'] = "['difficulty falling asleep']"
train.loc[6016, 'location'] = "['225 250']"

train.loc[6192, 'annotation'] = "['helps to take care of aging mother and in-laws']"
train.loc[6192, 'location'] = "['197 218;236 260']"

train.loc[6380, 'annotation'] = "['No hair changes', 'No skin changes', 'No GI changes', 'No palpitations', 'No excessive sweating']"
train.loc[6380, 'location'] = "['480 482;507 519', '480 482;499 503;512 519', '480 482;521 531', '480 482;533 545', '480 482;564 582']"

train.loc[6562, 'annotation'] = "['stressed due to taking care of her mother', 'stressed due to taking care of husbands parents']"
train.loc[6562, 'location'] = "['290 320;327 337', '290 320;342 358']"

train.loc[6862, 'annotation'] = "['stressor taking care of many sick family members']"
train.loc[6862, 'location'] = "['288 296;324 363']"

train.loc[7022, 'annotation'] = "['heart started racing and felt numbness for the 1st time in her finger tips']"
train.loc[7022, 'location'] = "['108 182']"

train.loc[7422, 'annotation'] = "['first started 5 yrs']"
train.loc[7422, 'location'] = "['102 121']"

train.loc[8876, 'annotation'] = "['No shortness of breath']"
train.loc[8876, 'location'] = "['481 483;533 552']"

train.loc[9027, 'annotation'] = "['recent URI', 'nasal stuffines, rhinorrhea, for 3-4 days']"
train.loc[9027, 'location'] = "['92 102', '123 164']"

train.loc[9938, 'annotation'] = "['irregularity with her cycles', 'heavier bleeding', 'changes her pad every couple hours']"
train.loc[9938, 'location'] = "['89 117', '122 138', '368 402']"

train.loc[9973, 'annotation'] = "['gaining 10-15 lbs']"
train.loc[9973, 'location'] = "['344 361']"

train.loc[10513, 'annotation'] = "['weight gain', 'gain of 10-16lbs']"
train.loc[10513, 'location'] = "['600 611', '607 623']"

train.loc[11551, 'annotation'] = "['seeing her son knows are not real']"
train.loc[11551, 'location'] = "['386 400;443 461']"

train.loc[11677, 'annotation'] = "['saw him once in the kitchen after he died']"
train.loc[11677, 'location'] = "['160 201']"

train.loc[12124, 'annotation'] = "['tried Ambien but it didnt work']"
train.loc[12124, 'location'] = "['325 337;349 366']"

train.loc[12279, 'annotation'] = "['heard what she described as a party later than evening these things did not actually happen']"
train.loc[12279, 'location'] = "['405 459;488 524']"

train.loc[12289, 'annotation'] = "['experienced seeing her son at the kitchen table these things did not actually happen']"
train.loc[12289, 'location'] = "['353 400;488 524']"

train.loc[13238, 'annotation'] = "['SCRACHY THROAT', 'RUNNY NOSE']"
train.loc[13238, 'location'] = "['293 307', '321 331']"

train.loc[13297, 'annotation'] = "['without improvement when taking tylenol', 'without improvement when taking ibuprofen']"
train.loc[13297, 'location'] = "['182 221', '182 213;225 234']"

train.loc[13299, 'annotation'] = "['yesterday', 'yesterday']"
train.loc[13299, 'location'] = "['79 88', '409 418']"

train.loc[13845, 'annotation'] = "['headache global', 'headache throughout her head']"
train.loc[13845, 'location'] = "['86 94;230 236', '86 94;237 256']"

train.loc[14083, 'annotation'] = "['headache generalized in her head']"
train.loc[14083, 'location'] = "['56 64;156 179']"

In [ ]:
print(train.loc[0])
print(train.loc[338])
print(train.loc[338, "annotation"])
print(train.loc[338, "location"])

print(train.loc[655])
print(train.loc[655, "annotation"])
print(train.loc[655, "location"])

train 데이터의 0번 index 한번 열어보기

In [ ]:
train[train["pn_num"] == 16]

annotation이 어떤 feature name인지 같이 보고싶음

In [ ]:
train_merge_feature = train.merge(features[['feature_num', 'feature_text']], on='feature_num')
train_merge_feature

pn_num이 16인거 한번 열어보기

In [ ]:
patient_notes.loc[16]

In [ ]:
patient_notes.loc[16, "pn_num"]

location의 from / to 한번 열어보기

In [ ]:
tmp = patient_notes.loc[16, "pn_history"]
print(tmp)
print('*' * 50)
tmp[696:724]

color visualization 해서 한번 보자.

In [ ]:
def print_colored_patient_note(train_data, feature_data, patient_data, pn_num):
    train_merge_feature = train.merge(features[['feature_num', 'feature_text']], on='feature_num')
    train_specific_patient = train_merge_feature[train_merge_feature["pn_num"] == pn_num]
    
    ents = []
    for i, row in train_specific_patient.iterrows():
        # print(row["location"])
        locs = literal_eval(row["location"])
        for loc in locs:
            # print(loc)
            begin = int(loc.split(' ')[0])
            end = int(loc.split(' ')[-1])
            ents.append({
                'start' : begin,
                'end' : end,
                'label' : row['feature_text']
            }
            )
            
    doc = {
        'text' : patient_data[patient_data["pn_num"] == pn_num]["pn_history"].iloc[0],
        "ents" : ents
    }

    # colors = {"Annotation" :"#33FAFA" } 
    colors = {} 
    options = {"colors": colors}
    spacy.displacy.render(doc, style="ent", options = options , manual=True, jupyter=True);
    
print_colored_patient_note(train, features, patient_notes, 16)

17yo, M 이런것도 유의미한 데이터 맞나..?

In [ ]:
test = pd.read_csv('../input/nbme-score-clinical-patient-notes/test.csv')
print(test.shape)
test

In [ ]:
sub = pd.read_csv('../input/nbme-score-clinical-patient-notes/sample_submission.csv')
sub

In [ ]:
tmp = patient_notes.loc[16, "pn_history"]
print(tmp[0:100])
print(tmp[200:250])
print(tmp[300:400])
print(tmp[75:110])

# CFG
일단 만들어둠

In [ ]:
class CFG:
    debug = False
    n_fold = 3
    model = "../input/huggingface-roberta-variants/roberta-large/roberta-large"
    dropout = 0.2
    batch_size = 4
    lr = 1e-5
    max_length = 480
    epochs = 5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Build Learning DataFrame
학습에 넣을 df 만들기

In [ ]:
def process_feature_text(text):
    text = re.sub('I-year', '1-year', text)
    text = re.sub('-OR-', " or ", text)
    text = re.sub('-', ' ', text)
    return text


def clean_spaces(txt):
    txt = re.sub('\n', ' ', txt)
    txt = re.sub('\t', ' ', txt)
    txt = re.sub('\r', ' ', txt)
    return txt

def build_learning_df(train_data, feature_data, patient_data):
    train_data["annotation_list"] = [literal_eval(x) for x in train["annotation"]]
    train_data["location_list"] = [literal_eval(x) for x in train["location"]]
    merged = train_data.merge(patient_data, how = "left")
    merged = merged.merge(feature_data, how = "left")
    merged = merged.loc[merged["annotation"] != "[]"].copy().reset_index(drop = True) # comment out if you train all samples
    
#     def process_feature_text(text):
#         return text.replace("-OR-", ";-").replace("-", " ")
#     merged["feature_text"] = [process_feature_text(x) for x in merged["feature_text"]]
    
    merged["feature_text"] = merged["feature_text"].apply(lambda x: x.lower())
    merged["pn_history"] = merged["pn_history"].apply(lambda x: x.lower())
    
    merged['pn_history'] = merged['pn_history'].apply(lambda x: x.strip())
    merged['feature_text'] = merged['feature_text'].apply(process_feature_text)
    merged['feature_text'] = merged['feature_text'].apply(clean_spaces)

    if CFG.debug:
        merged = merged.sample(frac = 0.5).reset_index(drop = True)

#     skf = StratifiedKFold(n_splits = CFG.n_fold)
#     merged["stratify_on"] = merged["case_num"].astype(str) + merged["feature_num"].astype(str)
#     merged["fold"] = -1
#     for fold, (_, valid_idx) in enumerate(skf.split(merged["id"], y = merged["stratify_on"])):
#         merged.loc[valid_idx, "fold"] = fold
        
    Fold = GroupKFold(n_splits=CFG.n_fold)
    groups = merged['pn_num'].values
    for n, (train_index, val_index) in enumerate(Fold.split(merged, merged['location'], groups)):
        merged.loc[val_index, 'fold'] = int(n)
    merged['fold'] = merged['fold'].astype(int)
    
    print(merged.shape)
    return merged

learning_df = build_learning_df(train, features, patient_notes)
learning_df

In [ ]:
def loc_list_to_ints(loc_list):
    to_return = []
    for loc_str in loc_list:
        loc_strs = loc_str.split(";")
        for loc in loc_strs:
            start, end = loc.split()
            to_return.append((int(start), int(end)))
    return to_return

print(learning_df.loc[338]["location_list"])
example_loc_ints = loc_list_to_ints(learning_df.loc[338]["location_list"])[0]
print(example_loc_ints)
print(learning_df.loc[338]["pn_history"][example_loc_ints[0] : example_loc_ints[1]])

# Tokenizer

In [ ]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
tokenizer.save_pretrained('tokenizer/')

In [ ]:
one_row = learning_df.loc[338]
one_row

In [ ]:
def tokenize_and_add_labels(tokenizer, one_row):
    tokenized_inputs = tokenizer(
        one_row["feature_text"],
        one_row["pn_history"],
        truncation = "only_second",
        max_length = CFG.max_length, # max length is 406
        padding = "max_length",
        return_offsets_mapping = True
    )
    labels = [0.0] * len(tokenized_inputs["input_ids"])
    tokenized_inputs["location_int"] = loc_list_to_ints(one_row["location_list"])
    tokenized_inputs["sequence_ids"] = tokenized_inputs.sequence_ids()

    for idx, (seq_id, offsets) in enumerate(zip(tokenized_inputs["sequence_ids"], tokenized_inputs["offset_mapping"])):
        if seq_id is None or seq_id == 0:
            labels[idx] = -100
            continue
            
        token_start, token_end = offsets
        for feature_start, feature_end in tokenized_inputs["location_int"]:
            if token_start >= feature_start and token_end <= feature_end:
                labels[idx] = 1.0
                break
                
    tokenized_inputs["labels"] = labels
    
    return tokenized_inputs

In [ ]:
tokenized_inputs = tokenize_and_add_labels(tokenizer, one_row)
for key in tokenized_inputs.keys():
    print(key)
    print(tokenized_inputs[key])
    print("=" * 100)

# Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        tokens = tokenize_and_add_labels(self.tokenizer, row)

        input_ids = np.array(tokens["input_ids"])
        attention_mask = np.array(tokens["attention_mask"])

        labels = np.array(tokens["labels"])
        offset_mapping = np.array(tokens['offset_mapping'])
        sequence_ids = np.array(tokens['sequence_ids']).astype("float16")
        
        return input_ids, attention_mask, labels, offset_mapping, sequence_ids

# Model

In [ ]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = AutoModel.from_pretrained(CFG.model)  # BERT model
        self.dropout = torch.nn.Dropout(p=CFG.dropout)
        self.fc1 = nn.Linear(1024, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 512)
        self.classifier = torch.nn.Linear(512, 1) # BERT has pooler_output(size: 768)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids = input_ids, attention_mask = attention_mask)[0]
        logits = self.fc1(outputs)
        logits = self.fc2(self.dropout(logits))
        logits = self.fc3(self.dropout(logits))
        logits = self.classifier(self.dropout(logits)).squeeze(-1)
        return logits

# Train & Evaluation

In [ ]:
from itertools import chain
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n = 1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def get_location_predictions(preds, offset_mapping, sequence_ids, test = False):
    all_predictions = []
    for pred, offsets, seq_ids in zip(preds, offset_mapping, sequence_ids):
        pred = sigmoid(pred)
        start_idx = None
        current_preds = []
        for p, o, s_id in zip(pred, offsets, seq_ids):
            if s_id is None or s_id == 0:
                continue
            if p > 0.5:
                if start_idx is None:
                    start_idx = o[0]
                end_idx = o[1]
            elif start_idx is not None:
                if test:
                    current_preds.append(f"{start_idx} {end_idx}")
                else:
                    current_preds.append((start_idx, end_idx))
                start_idx = None
        if test:
            all_predictions.append("; ".join(current_preds))
        else:
            all_predictions.append(current_preds)
    return all_predictions

def calculate_char_CV(predictions, offset_mapping, sequence_ids, labels):
    all_labels = []
    all_preds = []
    for preds, offsets, seq_ids, labels in zip(predictions, offset_mapping, sequence_ids, labels):
        num_chars = max(list(chain(*offsets)))
        char_labels = np.zeros((num_chars))
        for o, s_id, label in zip(offsets, seq_ids, labels):
            if s_id is None or s_id == 0:
                continue
            if int(label) == 1:
                char_labels[o[0]:o[1]] = 1
        char_preds = np.zeros((num_chars))
        for start_idx, end_idx in preds:
            char_preds[start_idx:end_idx] = 1
        all_labels.extend(char_labels)
        all_preds.extend(char_preds)
    results = precision_recall_fscore_support(all_labels, all_preds, average = "binary")
    return {
        "precision": results[0],
        "recall": results[1],
        "f1": results[2]
    }

def compute_metrics(p):
    predictions, y_true = p
    y_true = y_true.astype(int)
    y_pred = [
        [int(p > 0.5) for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, y_true)
    ]
    y_true = [
        [l for l in label if l != -100] for label in y_true
    ]
    results = precision_recall_fscore_support(list(chain(*y_true)), list(chain(*y_pred)), average = "binary")
    return {
        "token_precision": results[0],
        "token_recall": results[1],
        "token_f1": results[2]
    }

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
for fold in range(CFG.n_fold):
    torch.cuda.empty_cache()
    history = {"train": [], "valid": []}
    best_loss = np.inf
    
    # Folding
    val_fold = fold

    model = CustomModel().to(CFG.device)
    criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr = CFG.lr)

    train = learning_df.loc[learning_df["fold"] != val_fold].reset_index(drop = True)
    valid = learning_df.loc[learning_df["fold"] == val_fold].reset_index(drop = True)

    train_ds = CustomDataset(train, tokenizer)
    valid_ds = CustomDataset(valid, tokenizer)
    train_dl = torch.utils.data.DataLoader(train_ds, batch_size = CFG.batch_size, pin_memory = True, shuffle = True, drop_last = True)
    valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size = CFG.batch_size * 2, pin_memory = True, shuffle = False, drop_last = False)
    
    print("Train Fold :", fold)
    for epoch in range(CFG.epochs):
        print("Epoch :", epoch)
        
        #training
        model.train()
        train_loss = AverageMeter()
        pbar = tqdm(train_dl)
        # input_ids, attention_mask, labels, offset_mapping, sequence_ids
        for batch in pbar:
            optimizer.zero_grad()
            input_ids = batch[0].to(CFG.device)
            attention_mask = batch[1].to(CFG.device)
    #         print(input_ids)
    #         print(input_ids.shape)
    #         print(attention_mask)
    #         print(attention_mask.shape)
            labels = batch[2].to(CFG.device)
            offset_mapping = batch[3]
            sequence_ids = batch[4]
            logits = model(input_ids, attention_mask)
            loss_fct = torch.nn.BCEWithLogitsLoss(reduction = "none")
            loss = loss_fct(logits, labels)
            loss = torch.masked_select(loss, labels > -1).mean() # we should calculate at "pn_history"; labels at "feature_text" are -100 < -1
            loss.backward()
            optimizer.step()
            train_loss.update(val = loss.item(), n = len(input_ids))
            pbar.set_postfix(Loss = train_loss.avg)
        print("epoch :", epoch, "train loss :", train_loss.avg)
        history["train"].append(train_loss.avg)

        #evaluation
        model.eval()
        valid_loss = AverageMeter()
        with torch.no_grad():
            for batch in tqdm(valid_dl):
                input_ids = batch[0].to(CFG.device)
                attention_mask = batch[1].to(CFG.device)
                labels = batch[2].to(CFG.device)
                offset_mapping = batch[3]
                sequence_ids = batch[4]
                logits = model(input_ids, attention_mask)
                loss_fct = torch.nn.BCEWithLogitsLoss(reduction = "none")
                loss = loss_fct(logits, labels)
                loss = torch.masked_select(loss, labels > -1).mean()
                valid_loss.update(val = loss.item(), n = len(input_ids))
                pbar.set_postfix(Loss = valid_loss.avg)
        print("epoch :", epoch, "val loss :", valid_loss.avg)
        history["valid"].append(valid_loss.avg)

        # save model
        if valid_loss.avg < best_loss:
            print("model performance is improved. %f -> %f" % (best_loss, valid_loss.avg))
            print("Model saved")
            best_loss = valid_loss.avg
            torch.save(model.state_dict(), "nbme_%d.pth" % (fold))

    # EVALUATION
    model.load_state_dict(torch.load("nbme_%d.pth" % (fold), map_location = CFG.device))

    model.eval()
    preds = []
    offsets = []
    seq_ids = []
    lbls = []
    with torch.no_grad():
        for batch in tqdm(valid_dl):
            input_ids = batch[0].to(CFG.device)
            attention_mask = batch[1].to(CFG.device)
            labels = batch[2].to(CFG.device)
            offset_mapping = batch[3]
            sequence_ids = batch[4]
            logits = model(input_ids, attention_mask)
            preds.append(logits.cpu().numpy())
            offsets.append(offset_mapping.numpy())
            seq_ids.append(sequence_ids.numpy())
            lbls.append(labels.cpu().numpy())
    preds = np.concatenate(preds, axis = 0)
    offsets = np.concatenate(offsets, axis = 0)
    seq_ids = np.concatenate(seq_ids, axis = 0)
    lbls = np.concatenate(lbls, axis = 0)
    location_preds = get_location_predictions(preds, offsets, seq_ids, test = False)
    score = calculate_char_CV(location_preds, offsets, seq_ids, lbls)
    print("Fold %d CV score :" % (fold), score)

# Inference

In [ ]:
def build_test_df():
    merged = test.merge(patient_notes, how = "left")
    merged = merged.merge(features, how = "left")

#     def process_feature_text(text):
#         return text.replace("-OR-", ";-").replace("-", " ")
#     merged["feature_text"] = [process_feature_text(x) for x in merged["feature_text"]]
    merged["feature_text"] = [x for x in merged["feature_text"]]
    
    print(merged.shape)
    return merged

In [ ]:
class CustomTestData(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        example = self.data.loc[idx]
        tokenized = self.tokenizer(
            example["feature_text"],
            example["pn_history"],
            truncation = "only_second",
            max_length = 416,
            padding = "max_length",
            return_offsets_mapping = True
        )
        tokenized["sequence_ids"] = tokenized.sequence_ids()

        input_ids = np.array(tokenized["input_ids"])
        attention_mask = np.array(tokenized["attention_mask"])
        offset_mapping = np.array(tokenized["offset_mapping"])
        sequence_ids = np.array(tokenized["sequence_ids"]).astype("float16")

        return input_ids, attention_mask, offset_mapping, sequence_ids

In [ ]:
test_df = build_test_df()
test_ds = CustomTestData(test_df, tokenizer)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size = CFG.batch_size * 2, pin_memory = True, shuffle = False, drop_last = False)

all_preds = None
for fold in range(CFG.n_fold):
    model = CustomModel().to(CFG.device)
    model.load_state_dict(torch.load("nbme_%d.pth" % (fold), map_location = CFG.device))
    model.eval()
    
    preds = []
    offsets = []
    seq_ids = []
    with torch.no_grad():
        for batch in tqdm(test_dl):
            input_ids = batch[0].to(CFG.device)
            attention_mask = batch[1].to(CFG.device)
            offset_mapping = batch[2]
            sequence_ids = batch[3]
            logits = model(input_ids, attention_mask)
            preds.append(logits.cpu().numpy())
            offsets.append(offset_mapping.numpy())
            seq_ids.append(sequence_ids.numpy())

    preds = np.concatenate(preds, axis = 0)
    
    if all_preds is None:
        all_preds = np.array(preds).astype(np.float32)
    else:
        all_preds += np.array(preds).astype(np.float32)
        
    torch.cuda.empty_cache()

all_preds /= CFG.n_fold
all_preds = all_preds.squeeze()

offsets = np.concatenate(offsets, axis = 0)
seq_ids = np.concatenate(seq_ids, axis = 0)

# location_preds = get_location_predictions(preds, offsets, seq_ids, test = True)
location_preds = get_location_predictions(all_preds, offsets, seq_ids, test = True)
test["location"] = location_preds
test[["id", "location"]].to_csv("submission.csv", index = False)
pd.read_csv("submission.csv").head()